In [ ]:
#Data 
import numpy as np
import scipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost
snap = 99
df=pd.read_csv("halodata_tng100-1_{}th_logadded.csv".format(snap))
print(df.head())

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assume you have independent variables X and a dependent variable y
features = ['logSubhaloMassType0', 'logSubhaloMassType1', 'logSubhaloMassType4',
       'logSubhaloSpin', 'logSubhaloVmax', 'logSubhaloVmaxRad',
       'logSubhaloStarMetallicity', 'logSubhaloGasMetallicity',
       'logSubhaloSFR']
dependent_var = ['logSubhaloBHMass']
df_t = df[(df['logSubhaloMassType0'] > -90) & (df['logSubhaloMassType4'] > -90) &
   (df['SubhaloStarMetallicity'] > -90) & (df['SubhaloStarMetallicity'] > -90) & (df['SubhaloGasMetallicity'] > -90)]
X = df_t[features]
y = df_t[dependent_var]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Create and fit the linear regression model
reg = LinearRegression()
reg.fit(X_train, y_train.values.ravel())
# Print the coefficients of the model
print(reg.coef_, reg.intercept_)
print(reg.score(X, y))

# Make predictions using the model
y_pred = reg.predict(X_test)

from sklearn.metrics import mean_squared_error
# Flatten the predictions and actual values if necessary
actual_values_ln = y_test.values.flatten()  # Assuming y_test is a DataFrame
predicted_values_ln = y_pred.flatten()  # Flatten predictions array

# Compute the Mean Squared Error
mse = mean_squared_error(actual_values_ln, predicted_values_ln)
print("Mean Squared Error:", mse)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test.iloc[:, 0], y=y_pred, alpha=0.4)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--') # A line representing perfect predictions
plt.text(s=f'MSE: {mse:.5f}', x=y_test.min(), y=y_pred.max(), fontsize=20, bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Actual SubhaloBHMass',fontsize=20)
plt.ylabel('Predicted SubhaloBHMass',fontsize=20)
plt.xscale("log")
plt.yscale("log")
plt.text(s=f'MSE: {mse:.5f}', x=y_test.min(), y=y_pred.max(), fontsize=20, bbox=dict(facecolor='white', alpha=0.5))
plt.savefig('LinearRegression.jpg')
plt.show()


# Get coefficients
coefficients = reg.coef_
feature_names = X.columns
feature_importance = pd.DataFrame(coefficients, index=feature_names, columns=['Coefficient'])

# Sort features by their coefficient values
feature_importance.sort_values(by='Coefficient', ascending=False, inplace=True)

print(feature_importance)
# Plotting feature importance
plt.figure(figsize=(10, 8))
sns.barplot(x=feature_importance['Coefficient'], y=feature_importance.index)
plt.title('Feature Importance in Linear Regression')
plt.xlabel('Coefficient Magnitude')
plt.ylabel('Features')
plt.savefig('LinearReg.jpg')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
spt=np.ones_like(df_t['logSubhaloVmax'])*0.1
print(s)
plt.scatter(df_t['logSubhaloVmax'],df_t['logSubhaloBHMass'], s=spt)
plt.title('Feature Importance in Linear Regression',fontsize=15)
plt.xlabel('SubhaloVmax (log10(km/s))',fontsize=15)
plt.ylabel('SubhaloBHMass (log10[SubhaloBHMass/M_solar])',fontsize=15)
plt.savefig('SubhaloVmax.jpg')

plt.figure(figsize=(10, 6))
plt.scatter(df_t['logSubhaloMassType4'],df_t['logSubhaloBHMass'],s=spt)
plt.title('Feature Importance in Linear Regression',fontsize=15)
plt.xlabel('SubhaloMassType4  (log10[Star_Mass/M_solar])',fontsize=15)
plt.ylabel('SubhaloBHMass (log10[SubhaloBHMass/M_solar])',fontsize=15)
plt.savefig('SubhaloMassType4.jpg')

In [ ]:
import shap

explainer = shap.Explainer(reg, X_train)
shap_values = explainer(X_test)

# Visualize the first prediction's explanation
shap.plots.waterfall(shap_values[100])
shap.plots.beeswarm(shap_values)

importances = reg.feature_importances_
feature_importances = dict(zip(features, importances))

# Convert to DataFrame for easier handling
importances_df = pd.DataFrame({'Feature': features, 'Importance': importances})
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Visualize feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importances_df)
plt.title('Feature Importances in RandomForestRegressor',fontsize=16)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
forest_regressor = RandomForestRegressor(n_estimators=50, random_state=42, max_depth=12)

# Train the model
forest_regressor.fit(X_train, y_train.values.ravel())  # .values.ravel() to convert it to 1D array if y_train is a DataFrame

# Make predictions
y_pred = forest_regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
# Flatten the predictions and actual values if necessary
actual_values_rf = y_test.values.flatten()  # Assuming y_test is a DataFrame
predicted_values_rf = y_pred.flatten()  # Flatten predictions array

# Compute the Mean Squared Error
mse = mean_squared_error(actual_values_rf, predicted_values_rf)
print("Mean Squared Error:", mse)
# Plotting Actual vs. Predicted Values
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test.iloc[:, 0], y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Ideal line where actual = predicted
plt.text(s=f'MSE: {mse:.5f}', x=y_test.min(), y=y_pred.max(), fontsize=20, bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Actual SubhaloBHMass',fontsize=20)
plt.ylabel('Predicted SubhaloBHMass',fontsize=20)
plt.xscale("log")
plt.yscale("log")
plt.text(s=f'MSE: {mse:.5f}', x=y_test.min(), y=y_pred.max(), fontsize=20, bbox=dict(facecolor='white', alpha=0.5))
plt.savefig("RandomForest.jpg")
plt.show()


In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
max_depths = [2, 4, 8, 10, 12, 15, 20, 30]
mse = []
r2=[]
roc=[]
for i, max_depth in enumerate(max_depths):
    params = {
    "n_estimators": 50,
    "max_depth": max_depth,
    "min_samples_split": 5
    }
    forest_regressor = RandomForestRegressor(**params)

    # Train the model
    forest_regressor.fit(X_train, y_train.values.ravel()) 
    y_pred_test=forest_regressor.predict(X_test)
    
    mse += [mean_squared_error(y_test, y_pred_test)]
    r2 +=[r2_score(y_test, y_pred_test)]
    
plt.figure(figsize=(10, 6))
plt.plot(max_depths,mse)
plt.xlabel('Max Depth')
plt.ylabel('MSE')

plt.figure(figsize=(10, 6))
plt.plot(max_depths,r2)
plt.xlabel('Max Depth')
plt.ylabel('R2 score')

In [ ]:
import shap

explainer = shap.Explainer(forest_regressor, X_train)
shap_values = explainer(X_test)

# Visualize the first prediction's explanation
shap.plots.waterfall(shap_values[100])
shap.plots.beeswarm(shap_values)

In [ ]:
importances = forest_regressor.feature_importances_
feature_importances = dict(zip(features, importances))

# Convert to DataFrame for easier handling
importances_df = pd.DataFrame({'Feature': features, 'Importance': importances})
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Visualize feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importances_df)
plt.title('Feature Importances in RandomForestRegressor')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(max_depths,mse,'-o')
plt.xlabel('Max Depth')
plt.ylabel('MSE')

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Data scaling (optional but recommended)
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Convert to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_torch = torch.tensor(y_train_scaled, dtype=torch.float32)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_torch = torch.tensor(y_test_scaled, dtype=torch.float32)

# Create datasets
train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset = TensorDataset(X_test_torch, y_test_torch)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


from torch import nn
class RegressionNet(nn.Module):
    def __init__(self, input_size):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.output = nn.Linear(64, 1)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        return self.output(x)

# Initialize the model
model = RegressionNet(X_train.shape[1])


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")

train(model, train_loader, criterion, optimizer, epochs=100)

model.eval()
with torch.no_grad():
    predictions = model(X_test_torch).numpy()
    predictions = scaler_y.inverse_transform(predictions)  # Rescale to original scale

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.scatter(y_test, predictions, alpha=0.5, color='red', label='Predicted vs Actual')
plt.xlabel('Actual Values',fontsize=16)
plt.ylabel('Predicted Values',fontsize=16)
plt.title('Actual vs. Predicted Values',fontsize=16)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.legend()
plt.show()


In [ ]:
print(predictions[:,0])
print(y_test['logSubhaloBHMass'])
print(y_test.iloc[:, 0])



In [ ]:

from sklearn.metrics import mean_squared_error
# Flatten the predictions and actual values if necessary
actual_values = y_test.values.flatten()  # Assuming y_test is a DataFrame
predicted_values = predictions.flatten()  # Flatten predictions array

# Compute the Mean Squared Error
mse = mean_squared_error(actual_values, predicted_values)
print("Mean Squared Error:", mse)

plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions, alpha=0.5, color='blue', label='Predicted vs Actual')
plt.text(s=f'MSE: {mse:.5f}', x=y_test.min(), y=y_pred.max(), fontsize=20, bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Actual SubhaloBHMass (normalized)',fontsize=16)
plt.ylabel('Predicted SubhaloBHMass (normalized)',fontsize=16)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
#plt.legend()
plt.text(s=f'MSE: {mse:.5f}', x=y_test.min(), y=y_pred.max(), fontsize=20, bbox=dict(facecolor='white', alpha=0.5))
plt.savefig('NN.jpg')
plt.show()

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(random_state=42,max_depth=4)
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
mse_xgb = mean_squared_error(y_test, xgb_y_pred)

# Plotting Actual vs. Predicted Values
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test.iloc[:, 0], y=xgb_y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Ideal line where actual = predicted
#plt.title('Actual vs. Predicted Values for SubhaloBHMass',fontsize=20)
plt.text(s=f'MSE: {mse_xgb:.5f}', x=y_test.min(), y=y_pred.max(), fontsize=20, bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('Actual SubhaloBHMass',fontsize=20)
plt.ylabel('Predicted SubhaloBHMass',fontsize=20)
plt.xscale("log")
plt.yscale("log")
plt.savefig('xgb.jpg')
plt.show()

mse_xgb = mean_squared_error(y_test, xgb_y_pred)
print(mse_xgb)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
max_depths = [2, 4, 6, 8, 10, 12, 15, 20, 30]
mse = []
r2=[]
roc=[]
for i, max_depth in enumerate(max_depths):
    xgb = RandomForestRegressor(random_state=42,max_depth=max_depths[i])

    # Train the model
    xgb.fit(X_train, y_train.values.ravel()) 
    y_pred_test=xgb.predict(X_test)
    
    mse += [mean_squared_error(y_test, y_pred_test)]
    r2 +=[r2_score(y_test, y_pred_test)]
    
plt.figure(figsize=(10, 6))
plt.plot(max_depths,mse)
plt.xlabel('Max Depth')
plt.ylabel('MSE')

plt.figure(figsize=(10, 6))
plt.plot(max_depths,r2)
plt.xlabel('Max Depth')
plt.ylabel('R2 score')